<a href="https://colab.research.google.com/github/AbdelrahmanEssam1/Deep-Learning/blob/main/CNN_DailyMail_News_Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import ToolKits

In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


In [ ]:
import re
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer , Trainer , TrainingArguments

# Load Dataset

In [ ]:
train_df = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
val_df = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")

train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
train_df["article"][1]

'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a payment for setting up the meetings," federal prosecutors said in a statement. The complai

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          287113 non-null  object
 1   article     287113 non-null  object
 2   highlights  287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [ ]:
train_df = train_df.sample(n = 6000 , random_state = 42).reset_index(drop = True)
val_df = val_df.sample(n = 500 , random_state = 42).reset_index(drop = True)

# Preprocessing

In [ ]:
train_df['article'][0]

"By . Mia De Graaf . Britons flocked to beaches across the southern coast yesterday as millions look set to bask in glorious sunshine today. Temperatures soared to 17C in Brighton and Dorset, with people starting their long weekend in deck chairs by the sea. Figures from Asda suggest the unexpected sunshine has also inspired a wave of impromptu barbecues, with sales of sausages and equipment expected to triple those in April. Sun's out: Brighton beach was packed with Britons enjoying the unexpected sunshine to start the long weekend as temperatures hit 17C across the south coast . Although frost is set to hit the south tonight - with temperatures dropping to 1C - Britons stocking up for a barbecue will be in luck tomorrow, with forecasters predicting dry and sunny weather across southern England, southern Wales and the south Midlands. In Weymouth, Dorset, the sun came out in time for the town's annual kite festival, held on the beach. But the good weather has not been enjoyed by all as

In [ ]:
train_df['article'][1]

'A couple who weighed a combined 32st were shamed into slimming by their own family - during Christmas dinner. Margaret Gibson, 37, and her husband, James, 41, from Biddulph, Staffs, started piling on the pounds after the birth of their two children just over a decade ago. But after taunts during the festive feast - and a warning from James\'s doctor that he couldn\'t undergo a procedure because he would \'die on the operating table\' - the pair took action and have lost more than 7st between them. Wake up call: James, 41, and Margaret Gibson, 37, have shed a combined 7st in the last 11 months . By last Christmas Margaret tipped the scales at 12st 5lb and James weighed an unhealthy 20st. But the mother-of-two received a stark wake-up call when her father made a hurtful comment about her size. Margaret, a law firm secretary, said: \'I can\'t remember exactly what my father said but he made some sort of flippant comment to me about our appetite and I was really embarrassed. \'It was lite

In [ ]:
import html

def clean_text(examples):
    text = examples.replace('\xa0', ' ')
    text = html.unescape(text) #Decode HTML entities (like &pound; → £)
    text = re.sub(r"\s+([?.!,\'])" , r"\1", text)
    text = re.sub(r"\s+" , " " , text)
    text = re.sub(r'([?.!])\1+' , r'\1' , text) #Remove repeated punctuation ("!!", ".." , "??")
    text = text.strip()
    return text

train_df['article'] = train_df['article'].apply(clean_text)
train_df['highlights'] = train_df['highlights'].apply(clean_text)

val_df['article'] = val_df['article'].apply(clean_text)
val_df['highlights'] = val_df['highlights'].apply(clean_text)

train_df

,id,article,highlights
0,ed0fed726929c1eeabe6c390e47128dbb7d7a055,By. Mia De Graaf. Britons flocked to beaches a...,People enjoyed temperatures of 17C at Brighton...
1,023cd84001b33aed4ff0f3f5ecb0fdd2151cf543,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...
2,6a70a0d8d3ed365fe1df6d35f1587a8b9b298618,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...
3,b37204c13ea38b511265e41ac69fb12acfb63f85,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey. Most o...
4,c24e5805afd5145bc48410e876db91d44a06be5e,By. Daily Mail Reporter. PUBLISHED:. 12:53 EST...,The Xue Long had provided the helicopter that ...
...,...,...,...
5995,8eba070eb4263ba245af5e08fc81c6ef75f4290a,By. Chris Wheeler. Follow @@ChrisWheelerDM. an...,Evra is on the brink of a permanent move to It...
5996,71da5642a4ad9a866a6b71663852fda924d38bb8,There are buses and then there is the AutoTram...,The AutoTram Extra Grand is being trialled in ...
5997,181f18f687090cd04fb4758cc645ac986486d857,"Editor's note: Kathleen Kennedy Townsend, the ...","Townsend, Helmke: U.S. guns are helping fuel t..."
5998,5bc61f3373c2c726026b997932df6e975f2d6cfc,By. Daily Mail Reporter. PUBLISHED:. 15:51 EST...,At least 17 more people have been wounded afte...


# Tokenization

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5 models are prompt-based. For summarization, prefix my input with "summarize: ".

In [ ]:
def tokenize_func(examples):
    model_input = "summarize: " + examples['article']
    target = examples['highlights']

    inputs = tokenizer(model_input , max_length = 512 , truncation = True , padding = "max_length")
    targets = tokenizer(target , max_length = 128 , truncation = True , padding = "max_length")
    inputs['labels'] = targets['input_ids']

    return inputs

In [ ]:
train_df = train_df.apply(tokenize_func , axis = 1)
val_df = val_df.apply(tokenize_func , axis = 1)

In [ ]:
train_df[0]

{'input_ids': [21603, 10, 938, 5, 2133, 9, 374, 3, 4744, 9, 89, 5, 7834, 8057, 3, 89, 11863, 12, 9252, 640, 8, 7518, 4939, 4981, 38, 4040, 320, 356, 12, 3905, 157, 16, 18020, 18275, 469, 5, 12579, 15, 7, 3, 14266, 26, 12, 1003, 254, 16, 25080, 11, 6200, 2244, 6, 28, 151, 1684, 70, 307, 1851, 16, 3854, 6406, 57, 8, 2805, 5, 7996, 7, 45, 282, 26, 9, 3130, 8, 7544, 18275, 65, 92, 3555, 3, 9, 6772, 13, 3, 16260, 1167, 17, 76, 17621, 7, 6, 28, 1085, 13, 18567, 7, 11, 1277, 1644, 12, 12063, 273, 16, 1186, 5, 3068, 31, 7, 91, 10, 25080, 2608, 47, 7614, 28, 7834, 8057, 5889, 8, 7544, 18275, 12, 456, 8, 307, 1851, 38, 7902, 1560, 1003, 254, 640, 8, 3414, 4939, 5, 1875, 2515, 3481, 19, 356, 12, 1560, 8, 3414, 8988, 3, 18, 28, 7902, 14087, 12, 209, 254, 3, 18, 7834, 8057, 1519, 53, 95, 21, 3, 9, 17621, 56, 36, 16, 5851, 5721, 6, 28, 7555, 277, 3, 29856, 2192, 11, 13546, 1969, 640, 7518, 2789, 6, 7518, 10256, 11, 8, 3414, 24914, 7, 5, 86, 101, 63, 11975, 6, 6200, 2244, 6, 8, 1997, 764, 91, 16, 97,

# Fine Tuning Model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = "/content/t5_checkpoints",
    num_train_epochs = 6,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir="./logs",
    logging_steps = 50,
    save_steps = 500,
    eval_steps = 50,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_df,
    eval_dataset = val_df
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdelrhmanessam829 (abdelrhmanessam829-a-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,9.507000
100,8.082500
150,5.217800
200,2.591500
250,1.708800
300,1.464000
350,1.346000
400,1.312900
450,1.228500
500,1.223200


TrainOutput(global_step=4500, training_loss=1.410795236375597, metrics={'train_runtime': 1979.4514, 'train_samples_per_second': 18.187, 'train_steps_per_second': 2.273, 'total_flos': 4872304852992000.0, 'train_loss': 1.410795236375597, 'epoch': 6.0})

# Save and Load Model

In [ ]:
model.save_pretrained("/content/t5_model")
tokenizer.save_pretrained("/content/t5_model")

('/content/t5_model/tokenizer_config.json',
 '/content/t5_model/special_tokens_map.json',
 '/content/t5_model/spiece.model',
 '/content/t5_model/added_tokens.json')

In [ ]:
#Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/t5_model")
tokenizer = T5Tokenizer.from_pretrained("/content/t5_model")

# Summarization System

In [ ]:
device = model.device

def summarize_article(article):
  article = clean_text(article)
  inputs = tokenizer("summarize: " + article , return_tensors = "pt" , padding = "max_length" ,max_length = 512 , truncation = True)
  inputs = {key : value.to(device) for key , value in inputs.items()}

  output = model.generate(
      inputs['input_ids'],
      max_length = 150,
      num_beams = 4,
      early_stopping = True
  )

  summary = tokenizer.decode(output[0] , skip_special_tokens = True)

  return summary

In [ ]:
# Test with the sample input
sample_article = """Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking
to the skies, some experts are questioning if having such packed out planes is putting passengers at risk.
They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger.
More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week,
a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy
to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.
'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.
\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.'
But could crowding on planes lead to more serious issues than fighting for space in the overhead lockers,
crashing elbows and seat back kicking? Tests conducted by the FAA use planes with a 31 inch pitch,
a standard which on some airlines has decreased .
Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches .
Cynthia Corbertt, a human factors researcher with the Federal Aviation Administration, that it conducts tests on how quickly passengers can leave a plane.
But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased,
reported the Detroit News. The distance between two seats from one point on a seat to the same point on the seat behind it is known as the pitch.
While most airlines stick to a pitch of 31 inches or above, some fall below this. While United Airlines has 30 inches of space, Gulf Air economy seats have between 29 and 32 inches,
Air Asia offers 29 inches and Spirit Airlines offers just 28 inches. British Airways has a seat pitch of 31 inches, while easyJet has 29 inches,
Thomson's short haul seat pitch is 28 inches, and Virgin Atlantic's is 30-31."""

summary = summarize_article(sample_article)
print("Summary:", summary)

Summary: A U.S consumer advisory group set up by the Department of Transportation said this week that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. Many economy seats on United Airlines have 30 inches of space, while some airlines offer as little as 28 inches.
